## Running, Visualizing, and Analyzing Molecular Dynamics (MD) Simulations ##
In this notebook, you will use Python tools to play around with Molecular Dynamics (MD) simulations. In this example, we will prepare a system of interacting hexamers that may spontaneously self-assemble into hexameric sheets. The hexamers look like the following:

<div>
<img src="toy_hex.png" width="250"/>
</div>
   
where the cyan spheres are the body of the molecule and the pink spheres are virtual interaction sites that will allow each hexamer to bind to other hexamers. This interaction is attractive and has the following Gaussian functional form:

$$ U(r_{ij}) = -A \exp{(-B r_{ij})} $$

where $U$ is the potential energy in kcal/mol, $r_{ij}$ is the pair distance between sites in Angstroms, and $A$/$B$ are model parameters. 

All of our simulations will be performed using LAMMPS, an open-source MD package developed at Sandia National Lab.

## What am I looking at right now?
This is a "Jupyter notebook", a tool for using Python interactively. For example, you can quickly work with data, run simulations, visualize data, etc. then decide you want to see how your results might change if you change a parameter earlier in your workflow. If you've used Mathematica before, the idea behind a Jupyter notebook is very similar. You hit `shift-enter` or `shift-return` to execute the code in a "cell". 

**Beware:**
- the good thing about notebooks is that they let you interact with your data in very flexible ways
- the bad thing is that you can execute cells out of order and overwrite variables in ways you forget or didn't expect

If you're getting weird results, it's best to either do `Cell->Run All` at the top to reset the entire notebook, or if really needed, `Kernel->Restart`

Let us run our first cell (right below) by hitting `shift-enter` inside the cell. This cell will import Python modules that we will need for the rest of the exercise. Note that the left of the cell will have a "star" marker (i.e., "In \[*\]") when it is running and an integer (i.e., "In \[1\]")when it is done.

In [1]:
# Time to setup the notebook! This might take a few minutes to finish.
# We are installing modules/libraries that other folks have created
# These libraries contain functions that will use (so that we don't reinvent the wheel)

!pip install lammps ase nglview mdtraj

In [2]:
# We now need to import or "load" the things we just installed.

import numpy as np
import mdtraj as md
import nglview as ngl
from lammps import IPyLammps
from ase.io import read

### Preparing the LAMMPS simulation ###
In your folder, you have the following files: 

(1) **supercell.pdb** - this is a file format known as the 'Protein Data Bank' format and contains the initial coordinates of our system as well as topology information (names and indices of atoms, residues, etc.)

(2) **system.data** - this is an input file used by LAMMPS and contains the coordinates and topology information for a single hexamer, which we will later tile to form the complete system

(3) **input.setup** - this is an input file used by LAMMPS that determines most of the simulation parameters (force field definition, integration method, timestep, etc.)

All of these files are text files so you should feel free to view their contents using your favorite text editor.

In the next cell, we will prepare the LAMMPS simulation using our input files. We can further modify the simulation using the `lammps` module, as you will see.

In [7]:
from data.lj_params import LJ_PARAMS

temperature = 4000  # Kelvin
element = "Au"

L = IPyLammps() # this prepares the lammps Python object
# out = L.file("Au.setup") # this loads in the other simulation settings

L.command("units metal")
L.command("dimension 3")
L.command("boundary p p p")
L.command("atom_style atomic")
L.command(f"variable latparam equal {LJ_PARAMS[element]['cutoff']}")
L.command("variable size equal 4")
L.command("variable writefreq equal 100")
L.command("variable dt equal 0.001")
L.command(f"variable T equal {float(temperature)}")
L.command("lattice fcc ${latparam}")
L.command("region whole block 0 ${size} 0 ${size} 0 ${size}")
L.command("create_box 1 whole")
L.command("lattice fcc ${latparam} orient x 1 0 0 orient y 0 1 0 orient z 0 0 1")
L.command("create_atoms 1 region whole")
L.command(f"pair_style lj/cut {LJ_PARAMS[element]['cutoff']}")
L.command(f"pair_coeff * * {LJ_PARAMS[element]['epsilon']} {LJ_PARAMS[element]['sigma']}")
L.command("mass 1 196.97")
L.command("timestep ${dt}")
L.command("fix mynvt all nvt temp $T $T $(dt*1000)")
L.command("velocity all create $T 90909 rot yes mom yes dist gaussian")
L.command("dump mydump all atom ${writefreq} atom.lammpstrj")
L.command('variable ke_kilojoules equal "ke * 1.602e-19 / atoms * 6.022e23 / 1000"')
L.command("thermo_style custom step temp etotal ke pe v_ke_kilojoules")
L.command("thermo 1000")
L.command("write_data initial.data")
L.command("run 100000")

LAMMPS (7 Feb 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
LAMMPS output is captured by PyLammps wrapper
Total wall time: 0:00:09
Lattice spacing in x,y,z = 9.69298 9.69298 9.69298
Created orthogonal box = (0 0 0) to (38.77192 38.77192 38.77192)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 9.69298 9.69298 9.69298
Created 256 atoms
  using lattice units in orthogonal box = (0 0 0) to (38.77192 38.77192 38.77192)
  create_atoms CPU = 0.000 seconds
System init for write_data ...
Generated 0 of 0 mixed pair_coeff terms from geometric mixing rule
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 11.69298
  ghost atom cutoff = 11.69298
  binsize = 5.84649, bins = 7 7 7
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair lj/cut, perpetual
      attributes: half, newt

### Running the simulation ###

Now that the simulation is prepared, we can run the simulation. Let us run a short simulation of 25000 steps to see what happens. You'll see that a file called `md.dcd` will be created.

NOTE: This next cell make take a few minutes to run. You will know it is still busy if you see the star in "In \[*\]"

In [5]:
# L.run(25000) #run for 25000 steps, should take a few seconds to run
atoms = read("initial.data", format="lammps-data")
atoms.write("initial.gro", format="gromacs")

### Visualizing the trajectory ###

You'll see in the output of the previous block that a bunch of text was printed out. This is the "log" of the simulation and contains useful information. For our purposes, we can skip this and instead visualize the actual trajectory.

We will use `nglview` to display the trajectory. Once it loads, you can click play to see the atoms in motion!

In [6]:
traj = md.load("atom.lammpstrj", top="initial.gro")
view = ngl.show_mdtraj(traj)
view.clear_representations()
view.add_spacefill(selection="all", color="cyan", radius=0.05)
# view.add_representation('ball+stick', selection='all', color='pink', radius=0.03)
view.stage.set_parameters(**{
    "clipNear": 0, "clipFar": 100, "clipDist": 1,
    # percentages, start of fog and where on full effect
    # background color
    "backgroundColor": "black",
})
view.center()
# view.display(True)
view

NGLWidget(max_frame=1000)

### Observation debrief ###

What did you observe when you played those movies? Did you see aggregation of the hexamers into a cluster? What shape is the cluster?

Keep this behavior in mind because now we are going to adjust the model parameters to see what happens. You should change the $A$ parameter to be lower (e.g., 2.0 kcal/mol) or higher (e.g., 4.0 kcal/mol). To do so, go back to the top of the notebook and change the line that originally says:

`L.variable("GAUSSA equal 2.65")`

Note that the original simulation uses 2.65 kcal/mol. What do you think will happen in either scenario? Do you notice any changes in assembly behavior? 